# Assignment

In this assignment we will create a model for segmentation of tumor from abdominal CT images using custom loss function modifications to increase prediction sensitivity.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [ ]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries:

In [ ]:
import numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, models, losses, layers, optimizers
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about he KiTS Challenge can be found here: https://kits21.kits-challenge.org/. In this exercise, we will use this dataset to derive a model for slice-by-slice kidney segmentation. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/ct_kits`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [ ]:
# --- Download dataset
datasets.download(name='ct/kits')

# Training

### Stratified Sampling

Use the following code block to define a custom configuration dictionary to increase the sampling distribution of tumor (`lbl-crp-02`) up to 30%:

In [ ]:
# --- Configs dict to implement stratified sampling
configs = {
    'batch': {'size': 16},
    'sampling': {
        'lbl-crp-00': 0.4,
        'lbl-crp-01': 0.3,
        'lbl-crp-02': 0.3}}

# --- Prepare generators
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

In the assignment, feel free to experiment with different stratified sampling distributions. 

### Define the backbone model

Use the following cell block to define your backbone for the semantic segmentation task:

In [ ]:
# --- Define input
x = Input(shape=?, dtype='float32')

# --- Define model

# --- Define logits
logits = ...

# --- Create model
backbone = Model(inputs=x, outputs=logits)

### Define training model

Use the following cell block to start building your training model using the backbone network.

In [ ]:
# --- Define inputs
inputs = {?}

# --- Define model
logits = backbone(inputs['dat'])

### Custom loss function

In order to create a high sensitivity classifier for tumor segmentation, a combined weighted and masked loss strategy should be implemented. More specifically, the following weighting tensor should be create:

* class 0 (background; non-kidney): set `wgt` to 0
* class 1 (background; kidney): set `wgt` to 1
* class 2 (foreground; tumor): set `wgt` to positive value

In addition, recall that you will need to convert the three-class ground-truth label into a binarized target label (tumor or no tumor).

In [ ]:
# --- Create weights

# --- Create y_true (binarized ground-truth)

# --- Create loss

### Custom metrics

The goal of weighted and/or masked loss functions in this example is maximize the sensitivity for tumor prediction. Thus, in addition to a standard Dice score metric, we will additionally use foreground sensitivity to track overall model performance. Recall that to adjust the metrics to account for a custom weighted loss function, one must simply ignore predictions from masked regions (e.g., the model is required to predict accurate results in these regions). 

In [ ]:
def calculate_dsc(y_true, y_pred, weights=None, c=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      y_true : ground-truth label
      y_pred : predicted logits scores
           c : class to calculate DSC on
    
    """  
    pass

In [ ]:
def calculate_sen(y_true, y_pred, weights=None, c=1, **kwargs):
    """
    Method to implement sensitivity (recall) on raw cross-entropy logits

    """

In [ ]:
# --- Create metrics
dsc = calculate_dsc(?)
sen = calculate_sen(?)

Now, we are ready to create the `training` model and add the corresponding loss and accuracy tensors. 

In [ ]:
# --- Create model
training = Model(inputs=inputs, outputs=?)

# --- Add loss

# --- Add metrics

### Compile the model

Use the following cell block to compile your model with an appropriate optimizer. 

### In-memory data

To speed up training, consider loading all your model data into RAM memory:

In [ ]:
# --- Load data into memory for faster training
client.load_data_in_memory()

### Train the model

Use the following cell block to train your model.

# Evaluation

Based on the tutorial discussion, use the following cells to calculate model performance. The following metrics should be calculated:

* pixel-wise sensitivity (mean, median, 25th percentile, 75th percentile)
* Dice score coefficient (mean, median, 25th percentile, 75th percentile)

### Performance

The following minimum performance metrics must be met for full credit:

* median pixel-wise sensitivity: >0.70
* median Dice score coefficient: >0.70

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort sensitivity and Dice score statistics. There is no need to submit training performance accuracy.

# Submission

Use the following line to save your model for submission:

In [ ]:
# --- Serialize a model
backbone.save('./model.hdf5')

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.